# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기

## Face Detection(얼굴이라는 것을 발견) vs Face Recognition(얼굴을 통해서 누구의 얼굴인지 파악)

In [4]:
import mediapipe as mp
import cv2

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# model_selection: 0(가까운 거리), 1(5미터 이내의 먼 거리), min_detection_confidence: 얼굴인지 아닌지 임계치에 대한 확률
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            # 6개 특징: 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)

                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose = keypoints[2]
                mouth = keypoints[3]
                right_ear = keypoints[4]
                left_ear = keypoints[5]

                h, w, _ = image.shape   # height, width, channel
                right_eye = (int(right_eye.x * w), int(right_eye.y * h))
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose = (int(nose.x * w), int(nose.y * h))
                
                # 양 눈에 동그라미 그리기
                cv2.circle(image, right_eye, 50, (255, 0, 0), cv2.LINE_AA)
                cv2.circle(image, left_eye, 50, (255, 0, 0), cv2.LINE_AA)
                cv2.circle(image, nose, 50, (255, 0, 0), cv2.LINE_AA)

        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()